### 5. Generator-Evaluator:

In [13]:
from typing_extensions import TypedDict, Literal
from typing import Annotated, List
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
# from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

In [14]:
# from dotenv import load_dotenv
# load_dotenv()
# import os

# llm = ChatGroq(model="llama-3.3-70b-versatile")
llm = ChatOllama(model = "gemma3:270m")
response = llm.invoke("HI")
response

AIMessage(content='Hello there! How can I help you today?\n', additional_kwargs={}, response_metadata={'model': 'gemma3:270m', 'created_at': '2025-10-01T16:39:36.6440311Z', 'done': True, 'done_reason': 'stop', 'total_duration': 402128000, 'load_duration': 160591700, 'prompt_eval_count': 10, 'prompt_eval_duration': 35853700, 'eval_count': 12, 'eval_duration': 204818300, 'model_name': 'gemma3:270m'}, id='run--d3168711-fbe2-41df-b1f3-1f7f3485e2a8-0', usage_metadata={'input_tokens': 10, 'output_tokens': 12, 'total_tokens': 22})

In [15]:
class State(TypedDict):
    topic:str
    generated_joke: str
    feedback:str
    funny_or_not: str

class Feedback(BaseModel):
    grade:Literal['funny', 'not funny'] = Field(description="Whether the joke is funny or not")
    feedback:str=Field(description="If the joke is not funny, provide constructive feedback on how to make it funnier")


evaluator = llm.with_structured_output(Feedback)

In [16]:
def llm_call_generator(state:State):
    """Generates a joke based on the topic provided in the state."""
    if state.get('feedback'):
        msg = llm.invoke(f"Write a joke about {state['topic']} but take into account the feedback: {state['feedback']}")
    else:
        msg = llm.invoke(f"Write a joke about {state['topic']}")
        return({"generated_joke": msg.content})

def llm_call_evaluator(state:State):
    """Evaluates the generated joke and provides feedback."""
    msg = evaluator.invoke(f"Here is a joke: {state['generated_joke']}. Is it funny? If not, provide constructive feedback on how to make it funnier.")
    return({"funny_or_not": msg.grade, "feedback": msg.feedback})

# Conditional edge function to route back the logic to generator if the joke is not funny
def route_joke(state:State):
    """Returns back to the generator or to the end based upon the feedback from the evaluator."""

    if state['funny_or_not'] == 'not funny':
        return 'not_funny'
    elif state['funny_or_not'] == 'funny':
        return 'funny'

In [17]:
from langgraph.graph import StateGraph,START,END

builder = StateGraph(State)

builder.add_node("generator",llm_call_generator)
builder.add_node("evaluator",llm_call_evaluator)
builder.add_node("route_joke",route_joke)

builder.add_edge(START,"generator")
builder.add_edge("generator","evaluator")
builder.add_conditional_edges("evaluator",route_joke,{'not_funny':'generator', 'funny':END})
builder.add_edge("route_joke",END)

builder_graph = builder.compile()

In [28]:
from IPython.display import Markdown

response = builder_graph.invoke({'topic':'ice cream'})
Markdown(response['generated_joke'])

Why don't ice cream lovers go to the beach? 
... Because they're always getting ice cream!
